In [1]:
#these are commonly used 6 digit mpin passowrds
commonly_used_6_digit = {
    '000000', '111111', '222222', '333333', '444444', '555555', '666666', '777777', '888888', '999999',  # Repeating
    '123456', '234567', '345678', '456789', '567890','654321', '543210', '987654', '876543', '765432',
    '121212', '131313', '010101', '202020', '112233', '332211',  # Patterns
    '100000', '200000', '300000', '400000', '500000', '600000', '700000', '800000', '900000'
    '111222', '111333', '222333', '696969', '333444', '444555', '555666', '666777', '777888', '888999', '999000'
}

In [2]:
import datetime

In [3]:
def format_date_part(part):
    return str(part).zfill(2)  #formats dates to 2 digits

In [4]:
#this function returns the set of string derived from the demographics given by the user
def generate_demographic_patterns(date_as_string):
    patterns = set()
    if not date_as_string:
        return patterns
    try:
        date = datetime.datetime.strptime(date_as_string, '%Y-%m-%d')
    except ValueError:
        return patterns

    day = format_date_part(date.day)
    month = format_date_part(date.month)
    year_full = str(date.year)
    year_short = format_date_part(date.year % 100)

    patterns.update({
        f"{day}{month}{year_short}",  #DDMMYY
        f"{month}{day}{year_short}",  #MMDDYY
        f"{year_short}{month}{day}",  #YYMMDD
        f"{year_short}{day}{month}",  #YYDDMM
        f"{day}{month}{year_full[2:]}",  #DDMM + last 2 digits of year
        f"{year_full}",  #YYYY
        f"{year_full}{month}",  #YYYYMM
        f"{year_full}{day}",  #YYYYDD
        f"{month}{day}{year_full[2:]}",  #MMDDYY
    })

    return patterns

In [5]:
#this function checks against the demographics provided by the user
def check_mpin_strength(mpin, dob_self='', dob_spouse='', anniversary=''):
    reasons = []

    if not isinstance(mpin, str) or len(mpin) != 6 or not mpin.isdigit():
        return {'strength': "INVALID", 'reasons': ['format_is_invalid']}

    if mpin in commonly_used_6_digit:
        reasons.append('COMMONLY_USED')

    if mpin in generate_demographic_patterns(dob_self):
        reasons.append('DEMOGRAPHIC_DOB_SELF')

    if mpin in generate_demographic_patterns(dob_spouse):
        reasons.append('DEMOGRAPHIC_DOB_SPOUSE')

    if mpin in generate_demographic_patterns(anniversary):
        reasons.append('DEMOGRAPHIC_ANNIVERSARY')

    strength = "STRONG" if not reasons else "WEAK"
    return {'strength': strength, 'reasons': sorted(set(reasons))}

In [6]:
test_cases = [
    #commonly used
    {'mpin': '123456', 'dob_self': '1982-12-31', 'dob_spouse': '1993-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '111111', 'dob_self': '1995-03-15', 'dob_spouse': '1982-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '000000', 'dob_self': '1982-12-31', 'dob_spouse': '1989-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '654321', 'dob_self': '1995-03-15', 'dob_spouse': '2008-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '112233', 'dob_self': '1982-12-31', 'dob_spouse': '2003-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '121212', 'dob_self': '1985-05-10', 'dob_spouse': '1986-06-11', 'anniversary': '2000-12-12', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    {'mpin': '666666', 'dob_self': '1979-04-30', 'dob_spouse': '1981-08-15', 'anniversary': '2005-05-05', 'expected_output': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    #self DOB
    {'mpin': '010290', 'dob_self': '1990-02-01', 'dob_spouse': '1982-07-20', 'anniversary': '1982-12-31', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}},  
    {'mpin': '900201', 'dob_self': '1990-02-01', 'dob_spouse': '1985-07-20', 'anniversary': '1999-11-11', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}}, 
    {'mpin': '020190', 'dob_self': '1990-02-01', 'dob_spouse': '1980-07-20', 'anniversary': '1982-12-31', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF']}},  

    #spouse DOB
    {'mpin': '150388', 'dob_self': '1995-03-01', 'dob_spouse': '1988-03-15', 'anniversary': '1998-07-20', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SPOUSE']}}, 
    {'mpin': '880315', 'dob_self': '1995-03-15', 'dob_spouse': '1988-03-15', 'anniversary': '1998-07-20', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SPOUSE']}}, 

    #anniversary
    {'mpin': '250999', 'dob_self': '1998-07-20', 'dob_spouse': '1990-01-02', 'anniversary': '1999-09-25', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_ANNIVERSARY']}},  
    {'mpin': '990925', 'dob_self': '1990-01-02', 'dob_spouse': '1998-07-20', 'anniversary': '1999-09-25', 'expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_ANNIVERSARY']}}, 

    #strong
    {'mpin': '748291', 'dob_self': '1982-12-31', 'dob_spouse': '1983-01-01', 'anniversary': '2001-07-07', 'expected_output': {'strength': 'STRONG', 'reasons': []}},
    {'mpin': '865432', 'dob_self': '1970-04-05', 'dob_spouse': '1972-06-10', 'anniversary': '1998-09-25', 'expected_output': {'strength': 'STRONG', 'reasons': []}},
    {'mpin': '579134', 'dob_self': '1995-03-15', 'dob_spouse': '1988-07-20', 'anniversary': '2005-11-10', 'expected_output': {'strength': 'STRONG', 'reasons': []}},

    #multiple reasons
    {'mpin': '010190', 'dob_self': '1990-01-01', 'dob_spouse': '1991-02-02', 'anniversary': '1990-01-01','expected_output': {'strength': 'WEAK', 'reasons': ['DEMOGRAPHIC_DOB_SELF', 'DEMOGRAPHIC_ANNIVERSARY']}},

    #invalid
    {'mpin': '12345', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': {'strength': 'INVALID', 'reasons': ['format_is_invalid']}}, 
    {'mpin': 'ABCDEF', 'dob_self': '', 'dob_spouse': '', 'anniversary': '', 'expected_output': {'strength': 'INVALID', 'reasons': ['format_is_invalid']}},  
    {'mpin': '', 'dob_self': '', 'dob_spouse': '', 'anniversary': '','expected_output': {'strength': 'INVALID', 'reasons': ['format_is_invalid']}}, 
]


In [7]:
#this function runs the above defined cases and shows the expected and actual results
def run_tests():
    for i, test in enumerate(test_cases):
        mpin = test['mpin']
        dob_self = test['dob_self']
        dob_spouse = test['dob_spouse']
        anniversary = test['anniversary']
        expected_output = test['expected_output']
        expected_strength = expected_output['strength']
        expected_reasons = sorted(expected_output['reasons'])

        actual_result = check_mpin_strength(mpin, dob_self, dob_spouse, anniversary)
        actual_strength = actual_result['strength']
        actual_reasons = sorted(actual_result['reasons'])

        passed = (actual_strength == expected_strength) and (actual_reasons == expected_reasons)

        status = "PASSED" if passed else "FAILED"
        print(f"Test {i+1}: MPIN='{mpin}', DOB_Self='{dob_self}', DOB_Spouse='{dob_spouse}', Anniv='{anniversary}'")
        print(f"  Expected: Strength='{expected_strength}', Reasons={expected_reasons}")
        print(f"  Actual:   Strength='{actual_strength}', Reasons={actual_reasons}")
        print(f"  Status: {status}\n")


In [8]:
#run this block to give input
user_mpin = input("Please enter a 6-digit MPIN to check: ")
dob_self = input("Enter your birth date (YYYY-MM-DD): ")
dob_spouse = input("Enter your spouse's birth date (YYYY-MM-DD): ")
anniversary = input("Enter your Wedding Anniversary (YYYY-MM-DD): ")

result = check_mpin_strength(user_mpin,dob_self, dob_spouse, anniversary)

print(f"\nYour MPIN '{user_mpin}' is: {result['strength']}")
if result['reasons']:
    print(f"Reasons: {', '.join(result['reasons'])}")
else:
    print("MPIN is strong.")

Please enter a 6-digit MPIN to check:  170804
Enter your birth date (YYYY-MM-DD):  2004-08-17
Enter your spouse's birth date (YYYY-MM-DD):  2003-03-12
Enter your Wedding Anniversary (YYYY-MM-DD):  2025-06-04



Your MPIN '170804' is: WEAK
Reasons: DEMOGRAPHIC_DOB_SELF


In [9]:
#runs the above 20 test cases
run_tests()

Test 1: MPIN='123456', DOB_Self='1982-12-31', DOB_Spouse='1993-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 2: MPIN='111111', DOB_Self='1995-03-15', DOB_Spouse='1982-07-20', Anniv='2005-11-10'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 3: MPIN='000000', DOB_Self='1982-12-31', DOB_Spouse='1989-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 4: MPIN='654321', DOB_Self='1995-03-15', DOB_Spouse='2008-07-20', Anniv='2005-11-10'
  Expected: Strength='WEAK', Reasons=['COMMONLY_USED']
  Actual:   Strength='WEAK', Reasons=['COMMONLY_USED']
  Status: PASSED

Test 5: MPIN='112233', DOB_Self='1982-12-31', DOB_Spouse='2003-01-01', Anniv='2001-07-07'
  Expected: Strength='WEAK', Reasons=[

In [34]:
#this code successfully checks the given 6 digit mpin against user demographics and most common pins